# MG2 Time-Step Convergence Tests

## Component Being Tested

Atmospheric cloud microphysics

## Description

**[ What we are looking for here is a high-level description of the tests being performed. Sean's documentation in his notebook was largely about the code. What we are looking for is documentation about the tests. So I think what is here would largely be replaced. ]**

This notebook provides the capability to make temporal convergence plots of the MG2 microphysics. While the testing itself is not fully automated, this notebook produces plots and statistics that make it easier to determine the degree to which MG2 converges at approximately first order.

This module requires a few dependencies (listed in the [Import statements](#import-statements) section below), as well as a compatible data file. Such a file should be a history file produced by EAM or CAM, or a netCDF file containing data in the same format (e.g. dimension names and order, same units for variables, and so on). This file should contain all variables used as input to the MG2 microphysics, each in all caps and prefixed with `MG2IN_`. E.g. there will be a `MG2IN_T` variable, an `MG2IN_Q` variable, and so on. Aerosol-related variables `RNDST` and `NACON` have an extra dimension of size four, which is handled using number suffixes rather than by adding a dimension to the netCDF file. E.g. the variables `MG2IN_RNDST1` through `MG2IN_RNDST4` should be present. For a full list of input variables, see the [Initialization](#initialization) code below.

## Variable Definitions

**[ Any variables mentioned in the _Description_, or that get plotted, or that are necessary in some way, should be defined here. ]**

## Discussion

**[ What I have in mind here is a discussion of of the details of the test. For example, the _Description_ might present equations for convergence with respect to time step. Here, we might provide the list of time steps chosen, why they were chosen, and any caveats having to do with the choice. ]**

## Results

**[ I have taken much of the code from Sean's notebook and put it in the python module mg2.py. This module imports Sean's old module and encapsulates much of what he does so that readers can look at this from a high level.**

**Currently, I have a `convergence_test` class, which the user constructs, calls its `initialize()` method, and then its `run_tests()` method. Looking at other python scripts Sean has in his repo, we might want to change the class to `verification_test`, change the `initialize()` and `run_tests()` method names to be convergence-specific, and then add other `init` and `run` methods. ]**

### Setup

In [ ]:
# MG2 module (partially created with f2py)
import mg2

# Put plots in the notebook
%matplotlib inline

# Obtain the data file
HIST_FILE_NAME = "MG2_data_collection.cam.h1.0001-01-06-00000.nc"
HIST_URL = "https://drive.google.com/a/uw.edu/file/d/1HIARthNbpvxo3EAEQn1MK0aANXgYJOUP/view?usp=drive_web"
mg2.obtain_datafile(HIST_FILE_NAME, HIST_URL)

# Construct the convergence test object
mgncol = 128
ctest = mg2.convergence_test(HIST_FILE_NAME, mgncol)

# Total number of columns to use in plots.
total_columns = 2 * mgncol

# Timesteps to run out (the data points in the plot).
# Must divide final_time exactly (currently not tolerant of floating-point error).
timesteps = [0.25, 1., 2., 5., 10., 15., 30., 60., 75., 100., 150., 300.]

# Initialize the convergence tests
ctest.initialize(total_columns, timesteps)

### Main loop

This loop runs MG2 on a designated number of columns using a range of time step sizes, and creates the data arrays that will be plotted (or otherwise used). At the end of the run, the 2-norm estimate of error for each column will be placed in the `norms` variable, which is a dictionary indexed by variable name, containing arrays of estimated errors at particular columns and timestep sizes.

For instance, the 2-norm of error in variable `Q`, in column `c`, for the `i+1`-th timestep in the `timesteps` array, will be located at `norms['Q'][c,i]`.

In [ ]:
# Run the convergence tests
%timeit -n1 -r1 ctest.run_tests()

### Statistical Convergence Plots

This function provides the ability to track the convergence of a particular variable. In order to do this, we need to collapse the vector of errors across all columns into a single number. This is done by finding the error at a given percentile (e.g. the 50th percentile is the median error).

In [ ]:
ctest.plot_variable_convergence("QR", (100., 90., 50.))

### Column-tracking Convergence Plots

Rather than tracking errors at a given percentile in each timestep, we can instead use percentiles to find a set of columns at *one* timestep, and then produce convergence plots specific to that column.

In [ ]:
ctest.plot_column_variable_convergence("QR", (100., 90., 50.), 5.)

In [ ]:
ctest.plot_column_variable_convergence("QR", (100., 90., 50.), 100.)